In [1]:
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

# Simple Moving Average function
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

# SMA Cross Strategy class
class SmaCross(Strategy):
    n1 = 30
    n2 = 100

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()



In [2]:
import yfinance as yf

# Get BTC-USD data from yfinance
data = yf.download('BTC-USD')

# Code for running the backtest.
# Assumes the existence of `data` variable containing backtest data.
bt = Backtest(data, SmaCross)
stats = bt.run()


[*********************100%%**********************]  1 of 1 completed
/var/folders/t2/vmnkmyxs42b6h_fbsdjtsp0c0000gn/T/ipykernel_69552/2896767434.py:8: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, SmaCross)


In [3]:
#print(stats['_trades'].to_string())
stats


Start                     2014-09-17 00:00:00
End                       2024-08-09 00:00:00
Duration                   3614 days 00:00:00
Exposure Time [%]                   94.854772
Equity Final [$]                523724.184906
Equity Peak [$]                  617868.64975
Return [%]                        5137.241849
Buy & Hold Return [%]            13062.477822
Return (Ann.) [%]                   49.133317
Volatility (Ann.) [%]              128.567873
Sharpe Ratio                         0.382159
Sortino Ratio                        1.004463
Calmar Ratio                         0.521444
Max. Drawdown [%]                  -94.225502
Avg. Drawdown [%]                  -14.914778
Max. Drawdown Duration      727 days 00:00:00
Avg. Drawdown Duration       70 days 00:00:00
# Trades                                   32
Win Rate [%]                             50.0
Best Trade [%]                    1113.534101
Worst Trade [%]                    -64.725432
Avg. Trade [%]                    

In [4]:
bt.plot()

/Users/harvey/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/harvey/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p1309', ...)

In [6]:
# Define a range of values to test for each parameter
param_grid = {'n1': range(5, 60, 5), 'n2': range(10, 90, 5)}
# Run the optimization
res = bt.optimize(**param_grid)

# Print the best results and the parameters that lead to these results
print("Best result: ", res['Return [%]'])
print("Parameters for best result: ", res['_strategy'])

/Users/harvey/anaconda3/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/11 [00:00<?, ?it/s]

Best result:  6064.449881286621
Parameters for best result:  SmaCross(n1=25,n2=30)


In [7]:
res

Start                     2014-09-17 00:00:00
End                       2024-08-09 00:00:00
Duration                   3614 days 00:00:00
Exposure Time [%]                   98.782849
Equity Final [$]                616444.988129
Equity Peak [$]                 756394.003754
Return [%]                        6064.449881
Buy & Hold Return [%]            13062.477822
Return (Ann.) [%]                   51.608094
Volatility (Ann.) [%]              104.394129
Sharpe Ratio                         0.494358
Sortino Ratio                        1.226148
Calmar Ratio                          0.71159
Max. Drawdown [%]                  -72.524993
Avg. Drawdown [%]                  -11.686241
Max. Drawdown Duration      566 days 00:00:00
Avg. Drawdown Duration       59 days 00:00:00
# Trades                                  159
Win Rate [%]                        49.056604
Best Trade [%]                     265.605778
Worst Trade [%]                    -41.624433
Avg. Trade [%]                    

In [45]:
res['_equity_curve']

,Equity,DrawdownPct,DrawdownDuration
2014-09-17,1.000000e+04,0.000000,NaT
2014-09-18,1.000000e+04,0.000000,NaT
2014-09-19,1.000000e+04,0.000000,NaT
2014-09-20,1.000000e+04,0.000000,NaT
2014-09-21,1.000000e+04,0.000000,NaT
...,...,...,...
2023-10-17,2.887846e+06,0.279331,NaT
2023-10-18,2.879018e+06,0.281534,NaT
2023-10-19,2.918556e+06,0.271667,NaT
2023-10-20,3.015833e+06,0.247391,NaT
